In [12]:
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
def parser(x):
    return datetime.strptime(x , '%d/%m/%Y')
ColumnList=[]

series= read_csv('DP26-skus monthly sales (FILTERED2).csv', parse_dates=['Date'] , index_col='Date', date_parser=parser)
for columns in series:
    ColumnList.append(columns)
print (ColumnList)
print(series.head())
series.plot()
pyplot.show()

c:\users\admin\appdata\local\programs\python\python36-32\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


['1000403940202', "'1000403940203", "'1000403940206", "'1000600164107", "'1000600164110", "'1001600066003", "'1001600066005", "'10020000611502", "'1002000063302", "'1002000063305", "'1002000063306", "'1002000063307", "'1002003933701", "'1002003933702", "'1002003933704", "'1002003933705", "'10020044127401", "'10020044127402", "'10020044127403", "'1004200045501", "'1004200045503", "'1004400337902", "'1004400337905", "'1004400337906", "'1005200524703", "'1005200524705", "'1005200524708", "'1005204362301", "'1005204362302", "'1005204362303", "'1005204362304", "'1011604219701", "'1011604219703", "'1011604219704", "'1011604219706", "'1011604229801", "'1011604229802", "'1011604239901", "'2000100012001", "'2000100012006", "'2000100012009", "'2000100012010", "'2000100142206", "'2000100142210", "'2000100321306", "'2001100126401", "'2001100126405", "'2001100126406", "'2001100126407", "'2001100126408", "'2001100126409", "'2001100126410", "'2001100136901", "'2001100136903", "'2001100136904", "'2001

<Figure size 640x480 with 1 Axes>

In [13]:
model={}
ColName=[]
M_tripleExpo_mape=[]
M_tripleExpo_conf=[]
M_tripleExpo_sum_error=[]
M_tripleExpo_forecast_sum=[]
model['M_tripleExpo_mape']=M_tripleExpo_mape
model['M_tripleExpo_conf']=M_tripleExpo_conf
model['M_tripleExpo_absolute_error']=M_tripleExpo_sum_error
model['M_tripleExpo forecast sum']=M_tripleExpo_forecast_sum
model['ColName']=ColName

In [14]:
model2={}
M_tripleExpo_forecast=[]
SKU=[]
model2['M_tripleExpo_forecast']=M_tripleExpo_forecast
model2['SKU']=SKU

In [15]:
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (((np.sum(np.abs(y_true - y_pred))) / (np.sum(y_pred))) * 100)

In [ ]:
def movingavg(series,l,u):
    min = 999999999999999
    for w in range (l,u+1):
        result=[]
        for j in range(w):
            result.append(series[j])
        for i in range (w,len(series)):
            result.append((sum(series[i-w:i]))/w)
        mape=mean_absolute_percentage_error(series[24:],result[24:])
        #print result
        if (0<=mape<min):
            min=mape
            minW=w
            minr=result
        print ('Window-size {} - mape {}'.format(w,mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('Min mape is {} - window size {} - absolute error {} - forecast sum {}'.format(mape,minW,absolute_error,forecast_sum))
    movingavg_mape.append(min)
    movingavg_conf.append(minW)
    movingavg_sum_error.append(absolute_error)
    movingavg_forecast_sum.append(forecast_sum)
    movingavg_forecast.extend(minr)
    return minr
def exponential_smoothing(series, alphaL,alphaU):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    min = 999999999999999
    
    for alpha in range (alphal, alphau+1):
        result=[]
        result = [series[0]] # first value is same as series
        for n in range(1, (len(series))):
            result.append((alpha/10.0) * series[n-1] + (1 - (alpha/10.0)) * result[n-1])
        mape=mean_absolute_percentage_error(series[24:],result[24:])
        #print result
        if (0<=mape<min):
            min=mape
            minalpha=(alpha/10.0)
            minr=result
        print ('alpha {} - mape {}'.format((alpha/10.0),mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('best alpha {} - mape is {} - absolute error {} - forecast sum {}'.format(minalpha,min,absolute_error,forecast_sum))
    simpleExpo_mape.append(min)
    simpleExpo_conf.append(minalpha)
    simpleExpo_sum_error.append(absolute_error)
    simpleExpo_forecast_sum.append(forecast_sum)
    simpleExpo_forecast.extend(minr)
    return minr

def double_exponential_smoothing(series, alphaL, alphaU, betaL, betaU):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    min = 999999999999999
    
    for alpha in range (alphal, alphau+1):
        result=[]
        for beta in range (betal, betau+1):
            result = [series[0], series[0]]
            for n in range(2, len(series)):
                if n == 2:
                    level, trend = series[0], series[1] - series[0]
                if n <= len(series): # we are forecasting
                    value = series[n-1]
                last_level, level = level, (alpha/10.0)*value + (1-(alpha/10.0))*(level+trend)
                trend = (beta/10.0)*(level-last_level) + (1-(beta/10.0))*trend
                result.append(level+trend)
            mape=mean_absolute_percentage_error(series[24:],result[24:])
            #print result
            if (0<=mape<min):
                min=mape
                minalpha=(alpha/10.0)
                minbeta=(beta/10.0)
                minr=result
            print ('alpha {} - beta {} - mape {}'.format((alpha/10.0),(beta/10.0),mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('min alpha {} - min beta {} - mape {} - absolute error {} - forecast sum {}'.format(minalpha,minbeta,min,absolute_error,forecast_sum))
    DoubleExpo_mape.append(min)
    DoubleExpo_conf.append([minalpha,minbeta])
    DoubleExpo_sum_error.append(absolute_error)
    DoubleExpo_forecast_sum.append(forecast_sum)
    DoubleExpo_forecast.extend(minr)
    return minr

def M_initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen
def M_initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
def M_triple_exponential_smoothing(series, slenL, slenU, alphaL, alphaU, betaL, betaU, gammaL, gammaU, n_preds):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    gammal=int(10*gammaL)
    gammau=int(10*gammaU)
    min = 999999999999999
    min2=999999999999999
    for slen in range (slenL, slenU+1):
        try:
            for alpha in range (alphal, alphau+1):
                for beta in range (betal, betau+1):
                    for gamma in range (gammal, gammau+1):
                        result=[]
                        seasonals = M_initial_seasonal_components(series, slen)
                        for i in range(len(series)+n_preds):
                            if i == 0: # initial values
                                smooth = series[0]
                                trend = M_initial_trend(series, slen)
                                result.append(series[0])
                                continue
                            if i >= len(series): # we are forecasting
                                m = i - len(series) + 1
                                result.append((smooth + m*trend) * seasonals[i%slen])
                            else:
                                val = series[i-1]
                                last_smooth, smooth = smooth, (alpha/10.0)*(val/seasonals[i%slen]) + (1-(alpha/10.0))*(smooth+trend)
                                trend = (beta/10.0) * (smooth-last_smooth) + (1-(beta/10.0))*trend
                                seasonals[i%slen] = (gamma/10.0)*(val/smooth) + (1-(gamma/10.0))*seasonals[i%slen]
                                result.append((smooth+trend)*seasonals[i%slen])
                        mape=mean_absolute_percentage_error(series[24:],result[24:])
                        #print result
                        if (0<=mape<min):
                            min=mape
                            minalpha=(alpha/10.0)
                            minbeta=(beta/10.0)
                            mingamma=(gamma/10.0)
                            minr=result
                        print ('alpha {} - beta {} - gamma {} - mape {}'.format((alpha/10.0),(beta/10.0),(gamma/10.0),mape))
            print('best alpha {} - best beta {} - best gamma {} - mape {} - slen {}'.format(minalpha,minbeta,mingamma,min,slen))
        except:
            continue
        if (0<=min<min2):
            min2=min
            bestalpha=minalpha
            bestbeta=minbeta
            bestgamma=mingamma
            bestslen=slen
            bestr=minr
    absolute_error=(min2/100)*sum(bestr[24:])
    forecast_sum=sum(bestr[24:])
    print ('A {} - B {} - G {} - M {} - S {} - AE {} - FS {}'.format(bestalpha,bestbeta,bestgamma,min2,bestslen,absolute_error,forecast_sum))
    M_tripleExpo_mape.append(min2)
    M_tripleExpo_conf.append([bestalpha,bestbeta,bestgamma])
    M_tripleExpo_sum_error.append(absolute_error)
    M_tripleExpo_forecast_sum.append(forecast_sum)
    M_tripleExpo_forecast.extend(bestr)
    return bestr

def A_initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen
def A_initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
def A_triple_exponential_smoothing(series, slenL, slenU, alphaL, alphaU, betaL, betaU, gammaL, gammaU, n_preds):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    gammal=int(10*gammaL)
    gammau=int(10*gammaU)
    min = 999999999999999
    min2=999999999999999
    for slen in range (slenL, slenU+1):
        try:
            for alpha in range (alphal, alphau+1):
                for beta in range (betal, betau+1):
                    for gamma in range (gammal, gammau+1):
                        result=[]
                        seasonals = A_initial_seasonal_components(series, slen)
                        for i in range(len(series)+n_preds):
                            if i == 0: # initial values
                                smooth = series[0]
                                trend = A_initial_trend(series, slen)
                                result.append(series[0])
                                continue
                            if i >= len(series): # we are forecasting
                                m = i - len(series) + 1
                                result.append((smooth + m*trend) + seasonals[i%slen])
                            else:
                                val = series[i-1]
                                last_smooth, smooth = smooth, (alpha/10.0)*(val-seasonals[i%slen]) + (1-(alpha/10.0))*(smooth+trend)
                                trend = (beta/10.0) * (smooth-last_smooth) + (1-(beta/10.0))*trend
                                seasonals[i%slen] = (gamma/10.0)*(val-smooth) + (1-(gamma/10.0))*seasonals[i%slen]
                                result.append(smooth+trend+seasonals[i%slen])
                        mape=mean_absolute_percentage_error(series[24:],result[24:])
                        #print result
                        if (0<=mape<min):
                            min=mape
                            minalpha=(alpha/10.0)
                            minbeta=(beta/10.0)
                            mingamma=(gamma/10.0)
                            minr=result
                        print ('alpha {} - beta {} - gamma {} - mape {}'.format((alpha/10.0),(beta/10.0),(gamma/10.0),mape))
            print('best alpha {} - best beta {} - best gamma {} - mape {} - slen {}'.format(minalpha,minbeta,mingamma,min,slen))
        except:
            continue
        if (0<=min<min2):
            min2=min
            bestalpha=minalpha
            bestbeta=minbeta
            bestgamma=mingamma
            bestslen=slen
            bestr=minr
    absolute_error=(min2/100)*sum(bestr[24:])
    forecast_sum=sum(bestr[24:])
    print ('A {} - B {} - G {} - M {} - S {} - AE {} - FS {}'.format(bestalpha,bestbeta,bestgamma,min2,bestslen,absolute_error,forecast_sum))
    A_tripleExpo_mape.append(min2)
    A_tripleExpo_conf.append([bestalpha,bestbeta,bestgamma])
    A_tripleExpo_sum_error.append(absolute_error)
    A_tripleExpo_forecast_sum.append(forecast_sum)
    A_tripleExpo_forecast.extend(bestr)
    return bestr

In [17]:
def forecasting ():
    for i in range(len(ColumnList)):
        ColName.append(ColumnList[i])
        for ik in range(0,36):
            SKU.append(ColumnList[i])
        print (ColumnList[i])
        print (i)
        print ("")
        M_triple_exponential_smoothing(series[ColumnList[i]],2,14,0.1,0.9,0.1,0.9,0.1,0.9,0)

In [18]:
forecasting()

1000403940202
0

alpha 0.1 - beta 0.1 - gamma 0.1 - mape -954.4938778207814
alpha 0.1 - beta 0.1 - gamma 0.2 - mape -741.5022759539606
alpha 0.1 - beta 0.1 - gamma 0.3 - mape -2444.6765488159285
alpha 0.1 - beta 0.1 - gamma 0.4 - mape 144.35674837464998
alpha 0.1 - beta 0.1 - gamma 0.5 - mape 48.61652482633283
alpha 0.1 - beta 0.1 - gamma 0.6 - mape 44.79200791003325
alpha 0.1 - beta 0.1 - gamma 0.7 - mape 45.13950735984882
alpha 0.1 - beta 0.1 - gamma 0.8 - mape 45.86480021204334
alpha 0.1 - beta 0.1 - gamma 0.9 - mape 46.8042428596282
alpha 0.1 - beta 0.2 - gamma 0.1 - mape -866.0280362716666
alpha 0.1 - beta 0.2 - gamma 0.2 - mape -1336.8348985896166
alpha 0.1 - beta 0.2 - gamma 0.3 - mape 490.45460261151106
alpha 0.1 - beta 0.2 - gamma 0.4 - mape 114.73261722635488
alpha 0.1 - beta 0.2 - gamma 0.5 - mape 54.15822201097457
alpha 0.1 - beta 0.2 - gamma 0.6 - mape 46.85832215555353
alpha 0.1 - beta 0.2 - gamma 0.7 - mape 45.70426065310376
alpha 0.1 - beta 0.2 - gamma 0.8 - mape 46.279

alpha 0.2 - beta 0.7 - gamma 0.8 - mape 46.67133442856514
alpha 0.2 - beta 0.7 - gamma 0.9 - mape 47.400455486442944
alpha 0.2 - beta 0.8 - gamma 0.1 - mape -4807.422616531623
alpha 0.2 - beta 0.8 - gamma 0.2 - mape 258.8022894894923
alpha 0.2 - beta 0.8 - gamma 0.3 - mape -159.67024268420752
alpha 0.2 - beta 0.8 - gamma 0.4 - mape 72.59816115990759
alpha 0.2 - beta 0.8 - gamma 0.5 - mape 48.065516555908154
alpha 0.2 - beta 0.8 - gamma 0.6 - mape 47.75999543766656
alpha 0.2 - beta 0.8 - gamma 0.7 - mape 46.217302767063
alpha 0.2 - beta 0.8 - gamma 0.8 - mape 46.22293003367293
alpha 0.2 - beta 0.8 - gamma 0.9 - mape 47.413524132127016
alpha 0.2 - beta 0.9 - gamma 0.1 - mape 1245.327541332274
alpha 0.2 - beta 0.9 - gamma 0.2 - mape 692.1690538681023
alpha 0.2 - beta 0.9 - gamma 0.3 - mape -666.4365943556811
alpha 0.2 - beta 0.9 - gamma 0.4 - mape 93.55460806922687
alpha 0.2 - beta 0.9 - gamma 0.5 - mape 48.4048301103844
alpha 0.2 - beta 0.9 - gamma 0.6 - mape 45.69636711662234
alpha 0.2 

alpha 0.4 - beta 0.6 - gamma 0.5 - mape 44.712787804055914
alpha 0.4 - beta 0.6 - gamma 0.6 - mape 45.87997659526516
alpha 0.4 - beta 0.6 - gamma 0.7 - mape 47.095003917501074
alpha 0.4 - beta 0.6 - gamma 0.8 - mape 48.71028825174155
alpha 0.4 - beta 0.6 - gamma 0.9 - mape 50.51038133477256
alpha 0.4 - beta 0.7 - gamma 0.1 - mape 220.5956343630473
alpha 0.4 - beta 0.7 - gamma 0.2 - mape 74.92560433819769
alpha 0.4 - beta 0.7 - gamma 0.3 - mape 50.97423127186327
alpha 0.4 - beta 0.7 - gamma 0.4 - mape 116.97985301440197
alpha 0.4 - beta 0.7 - gamma 0.5 - mape 52.07305480260024
alpha 0.4 - beta 0.7 - gamma 0.6 - mape 38.97023208941248
alpha 0.4 - beta 0.7 - gamma 0.7 - mape 47.311930591902346
alpha 0.4 - beta 0.7 - gamma 0.8 - mape 49.177035547411286
alpha 0.4 - beta 0.7 - gamma 0.9 - mape 51.270686614303074
alpha 0.4 - beta 0.8 - gamma 0.1 - mape 275.2557349332346
alpha 0.4 - beta 0.8 - gamma 0.2 - mape -175.20953653672916
alpha 0.4 - beta 0.8 - gamma 0.3 - mape -189.3029680792593
alpha

alpha 0.6 - beta 0.7 - gamma 0.1 - mape 77.48776512246195
alpha 0.6 - beta 0.7 - gamma 0.2 - mape 76.43540610854261
alpha 0.6 - beta 0.7 - gamma 0.3 - mape 64.71537367796921
alpha 0.6 - beta 0.7 - gamma 0.4 - mape 51.197424202573906
alpha 0.6 - beta 0.7 - gamma 0.5 - mape 52.17454080131697
alpha 0.6 - beta 0.7 - gamma 0.6 - mape 53.40033056138097
alpha 0.6 - beta 0.7 - gamma 0.7 - mape 54.82794749053532
alpha 0.6 - beta 0.7 - gamma 0.8 - mape 56.13968689531421
alpha 0.6 - beta 0.7 - gamma 0.9 - mape 57.476726176896356
alpha 0.6 - beta 0.8 - gamma 0.1 - mape 110.11992947768736
alpha 0.6 - beta 0.8 - gamma 0.2 - mape 61.201638289046365
alpha 0.6 - beta 0.8 - gamma 0.3 - mape 95.78500646625895
alpha 0.6 - beta 0.8 - gamma 0.4 - mape 50.2596326476716
alpha 0.6 - beta 0.8 - gamma 0.5 - mape 54.3291771172892
alpha 0.6 - beta 0.8 - gamma 0.6 - mape 55.74028709231554
alpha 0.6 - beta 0.8 - gamma 0.7 - mape 56.97336101153249
alpha 0.6 - beta 0.8 - gamma 0.8 - mape 58.44448856575338
alpha 0.6 - 

alpha 0.8 - beta 0.6 - gamma 0.4 - mape 56.75785034940641
alpha 0.8 - beta 0.6 - gamma 0.5 - mape 56.11334660369014
alpha 0.8 - beta 0.6 - gamma 0.6 - mape 56.7655391640542
alpha 0.8 - beta 0.6 - gamma 0.7 - mape 57.19738131882025
alpha 0.8 - beta 0.6 - gamma 0.8 - mape 57.85668405923512
alpha 0.8 - beta 0.6 - gamma 0.9 - mape 57.38787107731213
alpha 0.8 - beta 0.7 - gamma 0.1 - mape 183.0903169355243
alpha 0.8 - beta 0.7 - gamma 0.2 - mape 49.13644215919131
alpha 0.8 - beta 0.7 - gamma 0.3 - mape 83.44222882718452
alpha 0.8 - beta 0.7 - gamma 0.4 - mape 58.19344983675175
alpha 0.8 - beta 0.7 - gamma 0.5 - mape 58.34095988569407
alpha 0.8 - beta 0.7 - gamma 0.6 - mape 58.210065875399366
alpha 0.8 - beta 0.7 - gamma 0.7 - mape 59.60857547448298
alpha 0.8 - beta 0.7 - gamma 0.8 - mape 58.57865946706426
alpha 0.8 - beta 0.7 - gamma 0.9 - mape 59.25126441334761
alpha 0.8 - beta 0.8 - gamma 0.1 - mape 51.21019501240327
alpha 0.8 - beta 0.8 - gamma 0.2 - mape 60.38972100251525
alpha 0.8 - be

...

alpha 0.6 - beta 0.2 - gamma 0.1 - mape -159.26222455494167
alpha 0.6 - beta 0.2 - gamma 0.2 - mape -190.7838623840558
alpha 0.6 - beta 0.2 - gamma 0.3 - mape -310.5313221926166
alpha 0.6 - beta 0.2 - gamma 0.4 - mape 1327.5855828357116
alpha 0.6 - beta 0.2 - gamma 0.5 - mape 416.297302416419
alpha 0.6 - beta 0.2 - gamma 0.6 - mape 93.77314900549314
alpha 0.6 - beta 0.2 - gamma 0.7 - mape -650.1904457977117
alpha 0.6 - beta 0.2 - gamma 0.8 - mape 258.0022126745422
alpha 0.6 - beta 0.2 - gamma 0.9 - mape 156.80355902853955
alpha 0.6 - beta 0.3 - gamma 0.1 - mape -187.5969478012295
alpha 0.6 - beta 0.3 - gamma 0.2 - mape -199.91873632520054
alpha 0.6 - beta 0.3 - gamma 0.3 - mape 286.57494005716313
alpha 0.6 - beta 0.3 - gamma 0.4 - mape -2053.287113948328
alpha 0.6 - beta 0.3 - gamma 0.5 - mape 278.9863950515282
alpha 0.6 - beta 0.3 - gamma 0.6 - mape 99.05473591247677
alpha 0.6 - beta 0.3 - gamma 0.7 - mape -673.4032737718107
alpha 0.6 - beta 0.3 - gamma 0.8 - mape 726.0322149535016
al

alpha 0.2 - beta 0.3 - gamma 0.7 - mape 222.27924398391056
alpha 0.2 - beta 0.3 - gamma 0.8 - mape 84.74237500577112
alpha 0.2 - beta 0.3 - gamma 0.9 - mape 260.1429217323153
alpha 0.2 - beta 0.4 - gamma 0.1 - mape 145.48910734451908
alpha 0.2 - beta 0.4 - gamma 0.2 - mape 140.33618946194431
alpha 0.2 - beta 0.4 - gamma 0.3 - mape 133.94337665239775
alpha 0.2 - beta 0.4 - gamma 0.4 - mape 125.67386664464857
alpha 0.2 - beta 0.4 - gamma 0.5 - mape 114.1473548645776
alpha 0.2 - beta 0.4 - gamma 0.6 - mape 94.1545485400495
alpha 0.2 - beta 0.4 - gamma 0.7 - mape 162.73176269183728
alpha 0.2 - beta 0.4 - gamma 0.8 - mape 77.0317556218224
alpha 0.2 - beta 0.4 - gamma 0.9 - mape 58.4039290966632
alpha 0.2 - beta 0.5 - gamma 0.1 - mape -255.3205829364615
alpha 0.2 - beta 0.5 - gamma 0.2 - mape -252.1731325210274
alpha 0.2 - beta 0.5 - gamma 0.3 - mape -250.3155885701859
alpha 0.2 - beta 0.5 - gamma 0.4 - mape -252.22961275280943
alpha 0.2 - beta 0.5 - gamma 0.5 - mape -286.8350598719734
alpha

alpha 0.4 - beta 0.7 - gamma 0.7 - mape 72.36056027108938
alpha 0.4 - beta 0.7 - gamma 0.8 - mape 99.67746733265895
alpha 0.4 - beta 0.7 - gamma 0.9 - mape 77.16450361854676
alpha 0.4 - beta 0.8 - gamma 0.1 - mape 771.3893980573085
alpha 0.4 - beta 0.8 - gamma 0.2 - mape 336.46877410979033
alpha 0.4 - beta 0.8 - gamma 0.3 - mape 133.27465140723095
alpha 0.4 - beta 0.8 - gamma 0.4 - mape 835.3512156926113
alpha 0.4 - beta 0.8 - gamma 0.5 - mape 171.84448084627192
alpha 0.4 - beta 0.8 - gamma 0.6 - mape 99.54781021815852
alpha 0.4 - beta 0.8 - gamma 0.7 - mape 92.40643726827096
alpha 0.4 - beta 0.8 - gamma 0.8 - mape 108.72913301567162
alpha 0.4 - beta 0.8 - gamma 0.9 - mape 631.1123773470258
alpha 0.4 - beta 0.9 - gamma 0.1 - mape 287.1576843491341
alpha 0.4 - beta 0.9 - gamma 0.2 - mape 231.04753472533983
alpha 0.4 - beta 0.9 - gamma 0.3 - mape 184.41173052955335
alpha 0.4 - beta 0.9 - gamma 0.4 - mape 143.79771231658344
alpha 0.4 - beta 0.9 - gamma 0.5 - mape 108.86060477705524
alpha 

alpha 0.9 - beta 0.6 - gamma 0.7 - mape -263.35561578167386
alpha 0.9 - beta 0.6 - gamma 0.8 - mape -215.2825229782994
alpha 0.9 - beta 0.6 - gamma 0.9 - mape -174.2842712073716
alpha 0.9 - beta 0.7 - gamma 0.1 - mape -258.48052200446216
alpha 0.9 - beta 0.7 - gamma 0.2 - mape -815.9586289894652
alpha 0.9 - beta 0.7 - gamma 0.3 - mape -220.43074936030274
alpha 0.9 - beta 0.7 - gamma 0.4 - mape -116.59887313386719
alpha 0.9 - beta 0.7 - gamma 0.5 - mape 131.2648705347901
alpha 0.9 - beta 0.7 - gamma 0.6 - mape 128.01362650019556
alpha 0.9 - beta 0.7 - gamma 0.7 - mape 140.42600368532885
alpha 0.9 - beta 0.7 - gamma 0.8 - mape 133.25219787851586
alpha 0.9 - beta 0.7 - gamma 0.9 - mape 110.1046474672968
alpha 0.9 - beta 0.8 - gamma 0.1 - mape -275.07075591902225
alpha 0.9 - beta 0.8 - gamma 0.2 - mape 1480.3673343322641
alpha 0.9 - beta 0.8 - gamma 0.3 - mape -2057.1174169424385
alpha 0.9 - beta 0.8 - gamma 0.4 - mape -1974.6521677535852
alpha 0.9 - beta 0.8 - gamma 0.5 - mape -1990.77248

alpha 0.7 - beta 0.5 - gamma 0.7 - mape 123.24389605514088
alpha 0.7 - beta 0.5 - gamma 0.8 - mape 53.09864133715559
alpha 0.7 - beta 0.5 - gamma 0.9 - mape -403.8027978509618
alpha 0.7 - beta 0.6 - gamma 0.1 - mape 107.1128517797082
alpha 0.7 - beta 0.6 - gamma 0.2 - mape 64.84545508386283
alpha 0.7 - beta 0.6 - gamma 0.3 - mape 59.67222908616504
alpha 0.7 - beta 0.6 - gamma 0.4 - mape 62.269526749252954
alpha 0.7 - beta 0.6 - gamma 0.5 - mape 69.62144541583208
alpha 0.7 - beta 0.6 - gamma 0.6 - mape -161.65141202193865
alpha 0.7 - beta 0.6 - gamma 0.7 - mape 147.8038596055744
alpha 0.7 - beta 0.6 - gamma 0.8 - mape 84.65015589647572
alpha 0.7 - beta 0.6 - gamma 0.9 - mape 98.13587208647196
alpha 0.7 - beta 0.7 - gamma 0.1 - mape 49.49123444247816
alpha 0.7 - beta 0.7 - gamma 0.2 - mape 217.9494933849832
alpha 0.7 - beta 0.7 - gamma 0.3 - mape 74.68992602538349
alpha 0.7 - beta 0.7 - gamma 0.4 - mape 193.8433641226612
alpha 0.7 - beta 0.7 - gamma 0.5 - mape 147.70039348463183
alpha 0.

In [20]:
print (model2)
df=pd.DataFrame(model2)
print (df)

{'M_tripleExpo_forecast': [9144.0, 175921.99923170495, 346176.4109702018, 208771.87138083292, 209092.88300298463, -655534.4485928776, 130447.33006893154, 111329.86125182807, 61546.5422655236, 64844.23930577182, -181265.36690742392, 41769.95541858675, 35845.50348458974, 23295.29631499212, 22553.57942861733, -45945.06737867649, 16814.927296161713, 15603.555008149451, 7737.66040060249, 8897.392883652214, -9014.5494689721, 6171.836383693514, 6887.494235205231, 5189.912191799201, 5691.3201441027495, 3336.825142734454, 2411.273355231728, 4393.134526948511, 4255.744712991485, 3318.6573143012033, 1340.4446256291549, 1103.8418906859672, 817.308184575062, 2327.2793110322627, 1693.8568742011914, 2577.032995887192, 9771.96, -171181.8558190338, 70827.23089706658, 15825.823262687238, -72305.1399373699, 58753.19982449799, -87557.19326051758, 54505.263179675705, -65974.99876763961, 39242.10495398459, -38593.583291218114, 25604.120237168983, -17498.422742833853, 15305.005430412897, -4758.01405250776, 9

In [21]:
print (model)
df1=pd.DataFrame(model)
print (df1)

{'M_tripleExpo_mape': [33.352300069637984, 19.89702400128449], 'M_tripleExpo_conf': [[0.9, 0.2, 0.1], [0.1, 0.4, 0.4]], 'M_tripleExpo_absolute_error': [11095.215970325115, 16916.270074960154], 'M_tripleExpo forecast sum': [33266.71907832096, 85019.09669440059], 'ColName': ['1000403940202', "'1000403940203"]}
          ColName  M_tripleExpo forecast sum  M_tripleExpo_absolute_error  \
0   1000403940202               33266.719078                 11095.215970   
1  '1000403940203               85019.096694                 16916.270075   

  M_tripleExpo_conf  M_tripleExpo_mape  
0   [0.9, 0.2, 0.1]          33.352300  
1   [0.1, 0.4, 0.4]          19.897024  
